In [9]:
# import TensorFlow as TF
import pandas as pd
import math
import matplotlib.pylab as plt
import numpy as np

from sklearn import datasets
from sklearn.impute import SimpleImputer # used for handling missing data

In [92]:

df = pd.read_csv('data.csv')
data_top = df.head()
copy = df.copy() 

# 1 - Remove rows where the target column (Scope1) is NaN.

copy = copy[copy['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'].notna()] # drop all rows with NaN in target column

copy = copy.reset_index() 



# 2 - Replace all NaN from the attribute columns and replace for mean of that column

#copy = copy.fillna(copy.mean())
#col_names = copy.columns
#sub2[col_names].fillna((sub2[col_names].mean()), inplace=True)

# 3 - Group rows by organization ID

# org_ids = list(copy['GTAP country'])
# list_set = set(org_ids)
# unique = list(list_set)

# copy = copy.groupby(['GTAP country'])

# dictionary = {} 

# for item in unique:
#     df_id = copy.get_group(item)
#     dictionary[item] = df_id


    
def create_dict(target_var, df): 
    org_ids = list(df[target_var])
    list_set = set(org_ids)
    unique = list(list_set)

    df = df.groupby([target_var])

    dictionary = {} 

    for item in unique:
        df_id = df.get_group(item)
        dictionary[item] = df_id
    
    return df_id
    
    
print(create_dict('GTAP sector', copy))



        index  organization_id StatementDetails.FinancialPeriodEndDate  \
57        182             6417                2010-12-31T00:00:00.000Z   
217       790             2014                2019-12-31T00:00:00.000Z   
266      1012             9235                2012-12-31T00:00:00.000Z   
281      1097             6398                2007-12-31T00:00:00.000Z   
375      1545             3832                2019-12-31T00:00:00.000Z   
...       ...              ...                                     ...   
24356   98174             2014                2016-12-31T00:00:00.000Z   
24379   98252            14625                2012-12-31T00:00:00.000Z   
24671   99441             1715                2007-12-31T00:00:00.000Z   
24983  100740             4325                2017-07-30T00:00:00.000Z   
25049  100983             4379                2019-12-31T00:00:00.000Z   

       StatementDetails.FinancialPeriodFiscalYear  \
57                                           2010   
217  